In [14]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

In [15]:
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [16]:
data_dir = '/kaggle/input/plantdisease/PlantVillage'
full_dataset = datasets.ImageFolder(data_dir, transform=data_transform)

In [17]:
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

In [18]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [19]:
num_classes = len(full_dataset.classes)
print(f"Dataset loaded successfully with {num_classes} classes.")
print(f"Training set has {len(train_dataset)} images.")
print(f"Validation set has {len(val_dataset)} images.")

Dataset loaded successfully with 15 classes.
Training set has 16510 images.
Validation set has 4128 images.


In [20]:
model = models.mobilenet_v2(weights='IMAGENET1K_V1')

In [21]:
for param in model.parameters():
    param.requires_grad = False

In [22]:
num_ftrs = model.classifier[1].in_features
model.classifier[1] = torch.nn.Linear(num_ftrs, num_classes)

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("MobileNetV2 model loaded and final layer replaced successfully.")
print(model.classifier)

MobileNetV2 model loaded and final layer replaced successfully.
Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=15, bias=True)
)


In [24]:
import torch.optim as optim
import torch.nn as nn

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
num_epochs = 10


In [ ]:
from tqdm import tqdm
for epoch in range(num_epochs):
    
    model.train()
    running_loss = 0.0
    
    train_progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Training]")
    
    for inputs, labels in train_progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_dataset)

    model.eval()
    val_loss = 0.0
    corrects = 0
    
    val_progress_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]")
    
    with torch.no_grad():
        for inputs, labels in val_progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            val_loss += loss.item() * inputs.size(0)
            corrects += torch.sum(preds == labels.data)

    epoch_val_loss = val_loss / len(val_dataset)
    epoch_val_acc = corrects.double() / len(val_dataset)

    print(f"Epoch {epoch+1}/{num_epochs} Summary | Train Loss: {epoch_loss:.4f} | Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_acc:.4f}\n")

print("Finished Training")

Epoch 1/10 [Validation]: 100%|██████████| 129/129 [00:25<00:00,  5.09it/s]


Epoch 1/10 Summary | Train Loss: 0.6343 | Val Loss: 0.2652 | Val Acc: 0.9312



Epoch 2/10 [Validation]: 100%|██████████| 129/129 [00:16<00:00,  7.84it/s]


Epoch 2/10 Summary | Train Loss: 0.3042 | Val Loss: 0.1925 | Val Acc: 0.9419



Epoch 3/10 [Validation]: 100%|██████████| 129/129 [00:16<00:00,  7.83it/s]


...

Epoch 8/10 [Validation]: 100%|██████████| 129/129 [00:16<00:00,  7.71it/s]


Epoch 8/10 Summary | Train Loss: 0.2007 | Val Loss: 0.1336 | Val Acc: 0.9549



Epoch 9/10 [Validation]: 100%|██████████| 129/129 [00:16<00:00,  7.68it/s]


Epoch 9/10 Summary | Train Loss: 0.1842 | Val Loss: 0.1342 | Val Acc: 0.9537



Epoch 10/10 [Validation]:  22%|██▏       | 28/129 [00:03<00:12,  8.02it/s]